In [32]:
#Need langchain and BEIR datasets
#!pip install langchain
#!pip install beir

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

2023-06-03 04:43:31 - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-06-03 04:43:31 - NumExpr defaulting to 8 threads.
/home/zainhas/anaconda3/envs/chunking/lib/python3.11/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-06-03 04:43:34 - Loading faiss with AVX2 support.
2023-06-03 04:43:34 - Successfully loaded faiss with AVX2 support.


In [ ]:
import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset = "nfcorpus"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(pathlib.Path('.').parent.absolute(), "datasets")
data_path = util.download_and_unzip(url, out_dir)


In [6]:
#### Provide the data_path where scifact has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2023-06-03 04:46:18 - Loading Corpus...


  0%|          | 0/3633 [00:00<?, ?it/s]

2023-06-03 04:46:18 - Loaded 3633 TEST Documents.
2023-06-03 04:46:18 - Doc Example: {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants die

In [13]:
corpus.keys()

dict_keys(['MED-10', 'MED-14', 'MED-118', 'MED-301', 'MED-306', 'MED-329', 'MED-330', 'MED-332', 'MED-334', 'MED-335', 'MED-398', 'MED-557', 'MED-666', 'MED-691', 'MED-692', 'MED-702', 'MED-706', 'MED-707', 'MED-708', 'MED-709', 'MED-711', 'MED-712', 'MED-713', 'MED-714', 'MED-716', 'MED-717', 'MED-718', 'MED-719', 'MED-720', 'MED-721', 'MED-722', 'MED-723', 'MED-724', 'MED-726', 'MED-727', 'MED-728', 'MED-729', 'MED-730', 'MED-731', 'MED-732', 'MED-733', 'MED-734', 'MED-735', 'MED-736', 'MED-743', 'MED-744', 'MED-745', 'MED-746', 'MED-748', 'MED-749', 'MED-751', 'MED-752', 'MED-753', 'MED-754', 'MED-756', 'MED-757', 'MED-758', 'MED-759', 'MED-760', 'MED-761', 'MED-762', 'MED-816', 'MED-818', 'MED-819', 'MED-820', 'MED-821', 'MED-822', 'MED-823', 'MED-824', 'MED-825', 'MED-826', 'MED-827', 'MED-828', 'MED-829', 'MED-830', 'MED-831', 'MED-832', 'MED-833', 'MED-834', 'MED-835', 'MED-836', 'MED-837', 'MED-838', 'MED-839', 'MED-840', 'MED-841', 'MED-842', 'MED-843', 'MED-844', 'MED-845', '

In [18]:
corpus['MED-10']['title']

'Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland'

In [17]:
corpus['MED-10']['text']

'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants died, of which 3,619 (60.2%) was due to breast cancer. After adjustment for age, tumor characteri

In [19]:
#Chunk the first document from NFC Corpus as an example

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap  = 50,
    length_function = len,
)

In [25]:
texts = text_splitter.create_documents([corpus['MED-10']['text']],metadatas=[{'document':'MED-10'}])
print(texts[0])

page_content='Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of' metadata={'document': 'MED-10'}


In [26]:
texts[0]

Document(page_content='Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of', metadata={'document': 'MED-10'})

In [27]:
texts[1]

Document(page_content='mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236', metadata={'document': 'MED-10'})

In [28]:
texts[2].page_content

'patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards'

In [30]:
queries['PLAIN-2']

'Do Cholesterol Statin Drugs Cause Breast Cancer?'

In [33]:
qrels['PLAIN-2']

{'MED-2427': 2,
 'MED-10': 2,
 'MED-2429': 2,
 'MED-2430': 2,
 'MED-2431': 2,
 'MED-14': 2,
 'MED-2432': 2,
 'MED-2428': 1,
 'MED-2440': 1,
 'MED-2434': 1,
 'MED-2435': 1,
 'MED-2436': 1,
 'MED-2437': 1,
 'MED-2438': 1,
 'MED-2439': 1,
 'MED-3597': 1,
 'MED-3598': 1,
 'MED-3599': 1,
 'MED-4556': 1,
 'MED-4559': 1,
 'MED-4560': 1,
 'MED-4828': 1,
 'MED-4829': 1,
 'MED-4830': 1}